In [1]:
import warnings
import os
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOpenAI


warnings.filterwarnings("ignore", category=DeprecationWarning) 
load_dotenv()
openAIApiKey = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(api_key=openAIApiKey)

# Agent

In [2]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

embeddings = OpenAIEmbeddings(api_key=openAIApiKey)
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)

e:\CodeProjects\notebooks\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
load_dotenv() 
tavilyKey = os.getenv("TAVILY_API_KEY")
search = TavilySearchResults(tavily_api_key=tavilyKey)

In [5]:
tools = [retriever_tool, search]

In [6]:
from langchain_community.chat_models import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "I want to go deeper into neuroscience. What is the current state of Artifical Synapse? What are main patent in that field?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current state of Artificial Synapse'}`


[{'url': 'https://onlinelibrary.wiley.com/doi/full/10.1002/aelm.202300889', 'content': 'The artificial synapse, a key element for learning and memory operations in neuromorphic computing, requires multi-state characteristics and the capability to change and store its states. The implementation of hardware-based artificial synapses using nonvolatile memory provides significant advantages in terms of energy consumption and circuit ...'}, {'url': 'https://onlinelibrary.wiley.com/doi/10.1002/smll.202100640', 'content': 'First, the fundamental synaptic behavior is reviewed, such as synaptic plasticity and learning rule, and requirements of artificial synapses to emulate biological synapses. In addition, an overview of recent advances on 2D materials-based synaptic devices is summarized by categorizing these into various working principles of artificial sy

{'input': 'I want to go deeper into neuroscience. What is the current state of Artifical Synapse? What are main patent in that field?',
 'output': "The current state of Artificial Synapse is quite advanced and is a key element for learning and memory operations in neuromorphic computing. It requires multi-state characteristics and the capability to change and store its states. The implementation of hardware-based artificial synapses using nonvolatile memory provides significant advantages in terms of energy consumption and circuit complexity. Recent advances have been made in 2D materials-based synaptic devices, categorized by various working principles of artificial synapses. Artificial synapses are memristor-based devices mimicking biological synapses, and they are used in neuromorphic computing systems that process information in a parallel, energy efficient way and store information in an analog, non-volatile form. There's also a focus on creating a hybrid self-activated artificial